<a href="https://colab.research.google.com/github/AMHlocal/final-project/blob/austin_branch/Tool_ReviewsNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import os
#latest version of spark 3.0 from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

#Install spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

#Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 256 kB in 9s (29.0 kB/s)
Reading package lists... Done


In [96]:
#Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReviews").getOrCreate()

In [97]:
#Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://cobrakaireviews.s3.us-east-2.amazonaws.com/amazon_reviews_us_Tools_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv"), sep="\t", header=True)

#Show DF
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [98]:
# count how many rows before dropping nulls
df.count()

1741100

In [99]:
# Find nulls in data
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               4|          4|            5|          5|   5|                5|              6|        121|         79|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [100]:
# Drop nulls in data
df1=df.dropna()
df1.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [101]:
# count number of rows after drop (should be less than 1741100(tools), 2402458(Grocery))
df1.count()

1740974

In [102]:
#Concatenate headline and body columns
from pyspark.sql import functions as sf
df2 = df1.withColumn('joined', sf.concat(sf.col('review_headline'), sf.lit(' '), sf.col('review_body')))
df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|Great organizer, ...|
|         US|   47910848|  RF0D1LEIF

### Feature Transformations


In [103]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
tokenizer = Tokenizer(inputCol='joined', outputCol='tokened')
stopremove = StopWordsRemover(inputCol='tokened', outputCol='stopped')
hashing = HashingTF(inputCol='stopped', outputCol='hashed', numFeatures=pow(2,4))
IDF = IDF(inputCol='hashed', outputCol='idf')

In [104]:
# tokenize review body
token = tokenizer.transform(df2)
token.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-0

In [105]:
#stopwords
stopped = stopremove.transform(token)
stopped.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|             stopped|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|    

In [106]:
#hashing dataframe
hashed_df = hashing.transform(stopped)
hashed_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|             stopped|              hashed|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...| 

In [107]:
# fit to IDF
idfModel = IDF.fit(hashed_df)
scaledData = idfModel.transform(hashed_df)
scaledData.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|             stopped|              hashed|                 idf|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         US|   15785389|

In [108]:
 #drop non-useful columns
reviews = scaledData.drop('marketplace', 'customer_id', 'review_id', 'product_parent', 'product_title', 'product_category', 
                          'vine', 'verified_purchase', 'review_date', 'helpful_votes', 'total_votes')
reviews.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B00H5U9ZD6|          5|Great organizer, ...|Very nice. Will o...|Great organizer, ...|[great, organizer...|[great, organizer...|(16,[1,4,5,8,11,1...|(16,[1,4,5,8,11,1...|
|B001TJGCS0|          4|Like it, but the ...|Like it, but the ...|Like it, but the ...|[like, it,, but, ...|[like, it,, batte...|(16,[0,1,2,3,4,5,...|(16,[0,1,2,3,4,5,...|
|B000NIK8JW|          1|        poor quality|We were excited t...|poor quality We w...|[poor, quality, w...|[poor, quality, e...|(16,[0,2,3,

In [109]:
# seeing if words are in dataframe and getting the count of the words
# common words: https://bit.ly/3mGZWEu, 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.functions import monotonically_increasing_id 


neg = ["broken", "broke", "brake", "dead", "died", "busted", "cracked", "ruin", "ruined", "failure", "failed", "chipped", "unstable", "defective", "shotty", "bent", "stop", "stopped", 		
	      "poor", "poorly", "risk", "risky", "bias", "bad", "sucks", "cheap", "tiny", "pain", "junk",
	      "replace", "rid", "back", "return", "tossed", "waste", "wasted", "highly", "too", "only",
       	"difficult", "adverse", "lack", "unclear", "negative", "wrong",
       	"hazard", "harm", "error", "severe", "exclusion", "hard", "absence", "loss", "falls",
	      "terrible", "disappointed", "disapointing", "horrible", "horribly", "ridiculously", "ridiculous", "inaccurate", "useless", "complex", "limitation", "limited", "confusing",
	      "junk", "crap", "shit", "shitty","sad",
	      "no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "hardly", "scarcely", "barely", "doesn’t", "isn’t", "wasn’t", "shouldn’t", "wouldn’t", "couldn’t", 
        "won’t", "can’t", "don’t", "doesnt", "didn't" ,"isnt", "wasnt", "shouldnt", "wouldnt", "couldnt", "wont", "cant", "dont", "didnt",
        "quit", "issue", "issues", "concerns", "concerned", "concern", "limitations", "problem", "problems", "hassle", "bother"]

def check(list):
  counter = 0
  for word in list:
    if word in neg:
      counter += 1
  return counter

def wordCount(list):
  return len(list)

def number(list):
  return int(list)

def ratio(list):
  return float(list)

pies = udf(lambda w : check (w), IntegerType())
apples = udf(lambda t : wordCount (t), IntegerType())
peach = udf(lambda e : number (e), IntegerType())
pear = udf(lambda f : ratio (f), FloatType())
zebra = reviews.withColumn('neg_count', pies(col('tokened')))
zebra = zebra.select("*").withColumn("id", monotonically_increasing_id())
zebra = zebra.select("*").withColumn("total_word_count", apples(col('tokened')))
zebra = zebra.select("*").withColumn("star_rating", peach(col('star_rating')))
zebra = zebra.select("*").withColumn("ratio", pear(col('neg_count')/col('total_word_count')))
# zebra = zebra[zebra['total_word_count'] > 10]
zebra.show()


+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+-----------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|neg_count| id|total_word_count|      ratio|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+-----------+
|B00H5U9ZD6|          5|Great organizer, ...|Very nice. Will o...|Great organizer, ...|[great, organizer...|[great, organizer...|(16,[1,4,5,8,11,1...|(16,[1,4,5,8,11,1...|        0|  0|              15|        0.0|
|B001TJGCS0|          4|Like it, but the ...|Like it, but the ...|Like it, but the ...|[like, it,, but, ...|[like, it,, batte...|(16,[0,1,2,

In [110]:
# smaller dataframe to work with
(panda, fish) = zebra.randomSplit([0.2, 0.8])
panda.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------+----------------+-----------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|neg_count|    id|total_word_count|      ratio|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------+----------------+-----------+
|1465799796|          5|          Five Stars|                Nice|     Five Stars Nice| [five, stars, nice]| [five, stars, nice]|(16,[2,6,10],[1.0...|(16,[2,6,10],[0.2...|        0|370006|               3|        0.0|
|1558706879|          3|         Three Stars|Good information ...|Three Stars Good ...|[three, stars, go...|[three, stars, go...

In [111]:
dogs = zebra.filter((zebra.id > 0) & (zebra.id < 2402212))
dogs.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+-----------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|neg_count| id|total_word_count|      ratio|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+-----------+
|B001TJGCS0|          4|Like it, but the ...|Like it, but the ...|Like it, but the ...|[like, it,, but, ...|[like, it,, batte...|(16,[0,1,2,3,4,5,...|(16,[0,1,2,3,4,5,...|        0|  1|              40|        0.0|
|B000NIK8JW|          1|        poor quality|We were excited t...|poor quality We w...|[poor, quality, w...|[poor, quality, e...|(16,[0,2,3,

In [112]:
features = dogs.drop("product_id", "star_rating", "review_headline", "review_body", "joined", "tokened", "stopped", "hashed", "id", "total_word_count")
features.show()

+--------------------+---------+-----------+
|                 idf|neg_count|      ratio|
+--------------------+---------+-----------+
|(16,[0,1,2,3,4,5,...|        0|        0.0|
|(16,[0,2,3,4,5,6,...|        5| 0.07936508|
|(16,[2,8,9,11,12,...|        0|        0.0|
|(16,[2,5,6,11,15]...|        2| 0.22222222|
|(16,[1,4,9,10,11,...|        0|        0.0|
|(16,[2,6,10,14,15...|        0|        0.0|
|(16,[3,4,9,10,11,...|        2|0.083333336|
|(16,[1,2,3,5,6,8,...|        0|        0.0|
|(16,[2,6,9],[0.25...|        0|        0.0|
|(16,[0,1,2,3,4,5,...|        1|0.008928572|
|(16,[0,1,2,3,4,5,...|        2|0.025316456|
|(16,[0,1,2,3,4,6,...|        1| 0.01923077|
|(16,[0,3,4,5,6,7,...|        1|0.020833334|
|(16,[1,2,5,6,8,10...|        0|        0.0|
|(16,[2,4,6,9,14,1...|        0|        0.0|
|(16,[0,1,2,3,4,5,...|       11|0.026829269|
|(16,[2,6,8],[0.25...|        0|        0.0|
|(16,[2,5,8,13],[0...|        0|        0.0|
|(16,[0,1,3,4,7,8,...|        3| 0.06382979|
|(16,[0,2,

In [113]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [114]:
output = assembler.transform(dogs).select('features','star_rating')

In [115]:
train,test = output.randomSplit([0.75, 0.25])

In [116]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(featuresCol='features', labelCol="star_rating")
print(type(dogs))
fitted_model = model.fit(train)
print("Coefficients: " + str(fitted_model.coefficients))
print("Intercept: " + str(fitted_model.intercept))

<class 'pyspark.sql.dataframe.DataFrame'>
Coefficients: [0.03685108154288445,0.005114004254973178,-0.07333202036725471,-0.025950579383392947,-0.024599279904208775,-0.012119280459400398,0.3119783243155745,-0.03238387394217759,0.005518397913141447,0.08175388678215546,0.007611977250839422,-0.032676691015895085,-0.028833536546359388,0.03175076032587236,0.12254786658636228,-0.04308657321893361,-0.22503410099908144,-7.3842334905419]
Intercept: 4.511313858472438


In [117]:
trainSummary = fitted_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("\nr2: %f" % trainSummary.r2)

RMSE: 1.039355

r2: 0.230523


In [118]:
from pyspark.sql.functions import abs
predictions = fitted_model.transform(test)
x = ((predictions['star_rating']- predictions['prediction'])/predictions['star_rating'])*100
predictions = predictions.withColumn('Accuracy', abs(x))
predictions.select("prediction","star_rating","Accuracy","features").show()

+------------------+-----------+------------------+--------------------+
|        prediction|star_rating|          Accuracy|            features|
+------------------+-----------+------------------+--------------------+
| 4.543743224882037|          1| 354.3743224882037|(18,[0],[0.880011...|
| 4.543743224882037|          4|13.593580622050915|(18,[0],[0.880011...|
| 4.543743224882037|          5| 9.125135502359267|(18,[0],[0.880011...|
| 4.543743224882037|          5| 9.125135502359267|(18,[0],[0.880011...|
| 4.543743224882037|          5| 9.125135502359267|(18,[0],[0.880011...|
| 4.543743224882037|          5| 9.125135502359267|(18,[0],[0.880011...|
| 4.576172591291636|          4|14.404314782290895|(18,[0],[1.760022...|
| 4.576172591291636|          5| 8.476548174167284|(18,[0],[1.760022...|
|4.5300574864607395|          4|13.251437161518487|(18,[0,1],[0.4400...|
| 4.532586431244241|          5| 9.348271375115171|(18,[0,1],[0.4400...|
| 4.532586431244241|          5| 9.348271375115171|

In [119]:
#r square for test dataset
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="star_rating",metricName="r2")
print("R Squared (R2) on test data = %g" % pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.230075


In [120]:
#adjusted r2
r2 = trainSummary.r2
n = dogs.count()
p = len(dogs.columns)
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.23049800803450637

In [ ]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='star_rating',maxIter=50, regParam=0.12, elasticNetParam=0.2)
linear_model = lin_reg.fit(train)
linear_model